In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from os import path

In [20]:
_DATA_DIR = path.join('/mnt', 'mnist')
_TEMP = path.join('/mnt', 'temp')

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('_DATA_DIR', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting _DATA_DIR/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting _DATA_DIR/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting _DATA_DIR/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting _DATA_DIR/t10k-labels-idx1-ubyte.gz


In [4]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [6]:
data.train.cls = np.argmax(data.train.labels, axis=1)

In [7]:
data.test.cls = np.argmax(data.test.labels, axis=1)

In [8]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 28

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

# Number of classes, one class for each of 10 digits.
num_classes = 10

In [9]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(data.train.images)},
    y=np.array(data.train.cls),
    num_epochs=None,
    shuffle=True)

In [10]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(data.test.images)},
    y=np.array(data.test.cls),
    num_epochs=1,
    shuffle=False)

In [16]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": some_images},
    num_epochs=1,
    shuffle=False)

In [18]:
def model_fn(features, labels, mode, params):
    # Args:
    #
    # features: This is the x-arg from the input_fn.
    # labels:   This is the y-arg from the input_fn,
    #           see e.g. train_input_fn for these two.
    # mode:     Either TRAIN, EVAL, or PREDICT
    # params:   User-defined hyper-parameters, e.g. learning-rate.
    
    # Reference to the tensor named "x" in the input-function.
    x = features["x"]

    # The convolutional layers expect 4-rank tensors
    # but x is a 2-rank tensor, so reshape it.
    net = tf.reshape(x, [-1, img_size, img_size, num_channels])    

    # First convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=16, kernel_size=5,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

    # Second convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv2',
                           filters=36, kernel_size=5,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)    

    # Flatten to a 2-rank tensor.
    net = tf.contrib.layers.flatten(net)
    # Eventually this should be replaced with:
    # net = tf.layers.flatten(net)

    # First fully-connected / dense layer.
    # This uses the ReLU activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc1',
                          units=128, activation=tf.nn.relu)    

    # Second fully-connected / dense layer.
    # This is the last layer so it does not use an activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc2',
                          units=10)

    # Logits output of the neural network.
    logits = net

    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)

        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        # Define the evaluation metrics,
        # in this case the classification accuracy.
        metrics = \
        {
            "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
        }

        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec

In [19]:
params = {"learning_rate": 1e-4}

In [21]:
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir=_TEMP)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/mnt/temp', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_tf_random_seed': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_master': '', '_save_checkpoints_steps': None, '_task_id': 0, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6c26d648d0>, '_task_type': 'worker', '_session_config': None, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_is_chief': True}


In [22]:
model.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /mnt/temp/model.ckpt.
INFO:tensorflow:loss = 2.31057, step = 1
INFO:tensorflow:global_step/sec: 201.197
INFO:tensorflow:loss = 0.969472, step = 101 (0.501 sec)
INFO:tensorflow:global_step/sec: 254.442
INFO:tensorflow:loss = 0.429319, step = 201 (0.390 sec)
INFO:tensorflow:global_step/sec: 262.335
INFO:tensorflow:loss = 0.318549, step = 301 (0.381 sec)
INFO:tensorflow:global_step/sec: 268.626
INFO:tensorflow:loss = 0.344466, step = 401 (0.372 sec)
INFO:tensorflow:global_step/sec: 264.632
INFO:tensorflow:loss = 0.315011, step = 501 (0.378 sec)
INFO:tensorflow:global_step/sec: 233.921
INFO:tensorflow:loss = 0.159646, step = 601 (0.433 sec)
INFO:tensorflow:global_step/sec: 217.411
INFO:tensorflow:loss = 0.26126, step = 701 (0.454 sec)
INFO:tensorflow:global_step/sec: 240.976
INFO:tensorflow:loss = 0.310163, step = 801 (0.417 sec)
INFO:tensorflow:global_step/sec: 266.534
INFO:tensorflow:loss = 0.150317

In [23]:
result = model.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Starting evaluation at 2018-04-06-08:57:01
INFO:tensorflow:Restoring parameters from /mnt/temp/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2018-04-06-08:57:02
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9777, global_step = 2000, loss = 0.073792


In [24]:
print("Classification accuracy: {0:.2%}".format(result["accuracy"]))

Classification accuracy: 97.77%


In [25]:
predictions = model.predict(input_fn=predict_input_fn)

In [26]:
cls_pred = np.array(list(predictions))
cls_pred

INFO:tensorflow:Restoring parameters from /mnt/temp/model.ckpt-2000


array([7, 2, 1, 0, 4, 1, 4, 9, 5])